In [2]:
import pandas as pd

df = pd.read_csv('dr_dr_se.csv')
df.head()

,drug_a,drug_b,side_effect_id,side_effect
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [3]:
import pandas as pd
import networkx as nx

df

G = nx.Graph()

# Add edges to the graph
for index, row in df.iterrows():
    G.add_edge(row['drug_a'], row['drug_b'], side_effect=row['side_effect'])

# Get the number of nodes and edges
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Print the number of nodes and edges
print(f"Number of nodes (drugs) in the graph: {num_nodes}")
print(f"Number of edges (drug interactions) in the graph: {num_edges}")

Number of nodes (drugs) in the graph: 645
Number of edges (drug interactions) in the graph: 63473


In [5]:
import pandas as pd
import numpy as np
from itertools import combinations

# Load your dataset
# Replace this with the path to your dataset
df = pd.read_csv("dr_dr_se.csv")

# Process existing data
# Creating a set of tuples for drug pairs with side effects
pairs_with_side_effects = set(df[['drug_a', 'drug_b']].apply(tuple, axis=1))

# Mark these pairs as 1 (having a side effect)
processed_data = pd.DataFrame(list(pairs_with_side_effects), columns=['drug_a', 'drug_b'])
processed_data['side_effect'] = 1

# Generate non-side effect data
all_drugs = set(df['drug_a']).union(set(df['drug_b']))
drug_combinations = set(combinations(all_drugs, 2))

# Filter out combinations that already have side effects
no_side_effect_combinations = drug_combinations - pairs_with_side_effects

# Convert to DataFrame and mark these as 0 (no side effect)
no_effect_data = pd.DataFrame(list(no_side_effect_combinations), columns=['drug_a', 'drug_b'])
no_effect_data['side_effect'] = 0

# Combine the data
final_dataset = pd.concat([processed_data, no_effect_data])

# Check for duplicates
final_dataset2 = final_dataset.drop_duplicates(subset=['drug_a', 'drug_b'])
final_dataset2
# Optionally, shuffle the dataset
#final_dataset = final_dataset.sample(frac=1).reset_index(drop=True)

# Save the processed dataset
final_dataset.to_csv("processed_dataset.csv", index=False)

In [6]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Example DataFrame (replace this with your actual DataFrame)
# df = pd.DataFrame({'drug_a': [...], 'drug_b': [...], 'interaction': [...]})

G = nx.Graph()
for _, row in final_dataset.iterrows():
    G.add_edge(row['drug_a'], row['drug_b'])  # Add an edge between the two drugs

In [7]:
pip install node2vec

     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
      --------------------------------------- 0.0/2.0 MB 653.6 kB/s eta 0:00:04
     ----- ---------------------------------- 0.3/2.0 MB 2.5 MB/s eta 0:00:01
     -------------------------- ------------- 1.4/2.0 MB 8.5 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
     ---------------------------------------- 67.1/67.1 kB ? eta 0:00:00
  Obtaining dependency information for simpful from https://files.pythonhosted.org/packages/8d/93/8448d3f1aa9d2911b8cba2602aaa1af85eb31a26d28b7b737f1fa5b40c02/simpful-2.11.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status '

In [8]:
from node2vec import Node2Vec
import networkx as nx

# Assuming 'G' is your NetworkX graph
node2vec = Node2Vec(G,
                    dimensions=64,
                    walk_length=40,
                    num_walks=50,
                    workers=2,
                    p=1,
                    q=0.5)

# Train Node2Vec Model
model = node2vec.fit(window=10,
                     min_count=1,
                     batch_words=4)

embeddings = {node: model.wv[node] for node in G.nodes()}

# Convert the embeddings dictionary to a list of tuples
embeddings_list = list(embeddings.items())

# Get the shape of the first node's embedding vector
# This assumes that all nodes have embeddings of the same shape
embedding_shape = embeddings_list[0][1].shape

print("The shape of the embedding vector is:", embedding_shape)

Computing transition probabilities:   0%|          | 0/645 [00:00<?, ?it/s]

The shape of the embedding vector is: (64,)


In [13]:
import numpy as np

# Example of combining Node2Vec embeddings with degree centrality
combined_features = {node: np.append(embeddings[node] for node in G.nodes())}

NameError: name 'node' is not defined

In [ ]:
# Assuming you have a DataFrame 'drug_pairs_df' with columns 'drug_a', 'drug_b', and 'has_side_effect'
feature_list = []
label_list = final_dataset['side_effect'].tolist()  # Target variable

for _, row in final_dataset.iterrows():
    # Concatenate the features of both drugs
    combined_feature = np.concatenate([combined_features[row['drug_a']], combined_features[row['drug_b']]])
    feature_list.append(combined_feature)

# Convert to numpy array
feature_matrix = np.array(feature_list)
labels = np.array(label_list)

In [ ]:
features_df = pd.DataFrame(feature_matrix)

# Add the labels as a new column in the DataFrame
features_df['label'] = labels

# Display the DataFrame
features_df

n_columns = features_df.shape[1]
features_df.columns = ['Vector_' + str(i) for i in range(1, n_columns + 1)]
features_df